In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from bs4 import BeautifulSoup
from time import sleep
import os
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_data_event(event):
    cleaned_event = re.sub('<tbody>|<tr>|<td>|</tbody>|</tr>|</td>|<td colspan="2">|\t','', event)
    nsymbols_data = 16
    date = cleaned_event[:nsymbols_data]
    cleaned_event_ = cleaned_event[nsymbols_data:].strip()
    splitted_event = cleaned_event_.split(';')
    players = splitted_event[0]
    try:
        cleaned_event_ = splitted_event[1].strip()
    except IndexError:
        cleaned_event_ = splitted_event[0].strip()
    if 'Победа' in cleaned_event:
        winner = cleaned_event_.split(',Тотал')[0].split(',  ')[1]
        splitted_event = cleaned_event_.split(', ')[0].split('  ')
        event_ = ' '.join(splitted_event[1:3])
        score = cleaned_event_.split('Итог матча:    ')[1].split(',  ')[0]
    else:
        winner = ''
        event_ = ' '.join(cleaned_event_.split('  ')[1:3])
        score = ''
    return date, event_, players, winner, score


def get_data_table(tables):
    page_tables = []
    for str_table in tables:
        soup = BeautifulSoup(str_table, 'html.parser')
        cleaned_table_data = [
            re.sub('<tr> | </tr>| class="c1"| class="c2"', '', str(x)) for x in soup.find_all('tr')[1:-1]
        ]
        columns = soup.find_all('th')
        columns = [re.sub('<th> | </th>', '', str(x)) for x in columns]
        table = []
        for str_ in cleaned_table_data: 
            soup_str = BeautifulSoup(str_, 'html.parser')
            table.append([x.text for x in soup_str.find_all('td')])
        table = pd.DataFrame(data=table, columns=columns)
        page_tables.append(table)
    return page_tables

# Calendar switcher

In [4]:
months = {
    'январь': '01',
    'февраль': '02',
    'март': '03',
    'апрель': '04',
    'май': '05',
    'июнь': '06',
    'июль': '07',
    'август': '08',
    'сентябрь': '09',
    'октябрь': '10',
    'ноябрь': '11',
    'декабрь': '12'
}

In [5]:
def save_data(i, current_page_month, current_page_year, data_tables, data_events):
    data_folder = '../data/'
    for bc in data_events.keys():
        # events saving
        # try:
        events_table = pd.DataFrame(
            np.vstack(data_events[bc]), 
            columns=['date', 'event', 'players', 'winner', 'score']
        )
        # except ValueError:
        #     continue
        bc_folder = bc.split('_')[1]
        file_name = f'{current_page_year}.{months[current_page_month.lower()]}.{i+1}'
        events_table.to_pickle(data_folder+'/'+bc_folder+'/events/'+file_name+'.pkl')

        # tables saving
        path = data_folder+'/'+bc_folder+'/tables/'+file_name
        if not os.path.exists(path):
            os.makedirs(path)
        # try:
        #     tables = np.hstack(np.array(data_tables[bc], dtype='object'))
        # except ValueError:
        #     j = 0 
        #     for table_list in data_tables[bc]:
        #         for table in table_list:
        #             table.to_pickle(path+'/'+str(j)+'.pkl')
        #             j += 1
        # else:
        #     for j, table in enumerate(tables):
        #         table.to_pickle(path+'/'+str(j)+'.pkl')
        for table_list in data_tables[bc]:
            j = 0 
            for table in table_list:
                table.to_pickle(path+'/'+str(j)+'.pkl')
                j += 1

In [6]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title
driver.maximize_window()
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [7]:
data_folder = '../data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
    for bc in bcs:
        os.makedirs(data_folder+'/'+bc.split('_')[1])
        os.makedirs(data_folder+'/'+bc.split('_')[1]+'/tables')
        os.makedirs(data_folder+'/'+bc.split('_')[1]+'/events')
    

In [8]:
# previous_date = dt.datetime.today() - dt.timedelta(days=1)
# prev_day = previous_date.day

# bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
# current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
# current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text
# first_month_flg = True
# # first_month_flg = False
# while current_page_month != 'Декабрь' or current_page_year != '2019':
#     dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#     n_range = range(len(dates_objects)-1, -1, -1)
#     # Date switcher
#     for i in n_range:
#         # skip first pages without data
#         if first_month_flg and (i+1 > prev_day):
#             continue
#         dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#         try:
#             dates_objects[i].click()
#         except:
#             dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#             # dates_objects[i].click()
#             element = dates_objects[i]
#             driver.execute_script("arguments[0].click();", element)
#         # objects with data for current date
#         data_events = {bc: [] for bc in bcs}
#         data_tables = {bc: [] for bc in bcs}
#         # BC switcher
#         for bc in bcs:
#             sleep(1)
#             try:
#                 driver.find_elements(By.ID, bc)[0].click()
#             except ElementClickInterceptedException:
#                 element = driver.find_elements(By.ID, bc)[0]
#                 driver.execute_script("arguments[0].click();", element)
#             sleep(1)
#             npages = driver.find_elements(By.CLASS_NAME, 'pages')
#             len_pages = len(npages)
#             if not len_pages:
#                 len_pages = 1
#             # pages iterating
#             for j in range(len_pages):
#                 if len_pages > 1:
#                     try:
#                         driver.execute_script("arguments[0].click();", npages[j])
#                     except (StaleElementReferenceException, WebDriverException):
#                         npages = driver.find_elements(By.CLASS_NAME, 'pages')
#                         driver.execute_script("arguments[0].click();", npages[j])    
#                 sleep(1)
#                 html_source_code = driver.execute_script("return document.body.innerHTML;")
#                 html_soup = BeautifulSoup(html_source_code, 'html.parser')
#                 tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
#                 # events parsing
#                 events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
#                 events = tbody_data[events_indexes]
#                 data_events[bc].append([get_data_event(x) for x in events])
#                 # tables parsing
#                 tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
#                 data_tables[bc].append(get_data_table(tables))
#         save_data(i, current_page_month, current_page_year, data_tables, data_events)
#     first_month_flg = False
#     # month switcher
#     left_triangle = driver.find_elements(By.LINK_TEXT, 'Назад')
#     left_triangle[0].click()
#     current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
#     current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text

In [9]:
# # pages iterating
# previous_date = dt.datetime.today() - dt.timedelta(days=1)
# prev_day = previous_date.day
# bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
# current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
# current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text
# first_month_flg = True

# data_events = {bc: [] for bc in bcs}
# data_tables = {bc: [] for bc in bcs}
# attributes = []
# while current_page_month != 'Декабрь' or current_page_year != '2019':
#     dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#     n_range = range(len(dates_objects)-1, -1, -1)
#     for i in n_range:
#         # skip first pages without data
#         if first_month_flg and (i+1 > prev_day):
#             continue
#         dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#         try:
#             dates_objects[i].click()
#         except:
#             dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#             element = dates_objects[i]
#             driver.execute_script("arguments[0].click();", element)
#         # BC switcher
#         for bc in bcs:
#             sleep(1)        
#             try:
#                 driver.find_elements(By.ID, bc)[0].click()
#             except ElementClickInterceptedException:
#                 element = driver.find_elements(By.ID, bc)[0]
#                 driver.execute_script("arguments[0].click();", element)
#             sleep(1)
#             buttons = WebDriverWait(driver, 10).until(
#                 EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".pages_pag .pages"))
#             )
#             filtered_ids = [x.get_attribute('id') for x in buttons if bc.split('_')[1] in x.get_attribute('id')]
#             # pages iterating
#             for id_ in filtered_ids:
#                 button = driver.find_elements(By.ID, id_)[0]
#                 try:
#                     driver.execute_script("arguments[0].click();", button) 
#                 except (StaleElementReferenceException, WebDriverException):
#                     sleep(1)
#                     button = driver.find_elements(By.ID, id_)[0]
#                     driver.execute_script("arguments[0].click();", button) 
#                 sleep(1)
#                 html_source_code = driver.page_source
#                 html_soup = BeautifulSoup(html_source_code, 'html.parser')
#                 sleep(1)
#                 tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
#                 # events parsing
#                 events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
#                 events = tbody_data[events_indexes]
#                 data_events[bc].append([get_data_event(x) for x in events])
#                 # tables parsing
#                 tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
#                 data_tables[bc].append(get_data_table(tables))
#         save_data(i, current_page_month, current_page_year, data_tables, data_events)
#     first_month_flg = False
#     # month switcher
#     left_triangle = driver.find_elements(By.LINK_TEXT, 'Назад')
#     left_triangle[0].click()
#     current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
#     current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text

In [ ]:
# парсинг только первых страниц для каждой БК
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text
first_month_flg = True

data_events = {bc: [] for bc in bcs}
data_tables = {bc: [] for bc in bcs}
attributes = []
while current_page_month != 'Декабрь' or current_page_year != '2019':
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    n_range = range(len(dates_objects)-1, -1, -1)
    for i in n_range:
        # skip first pages without data
        if first_month_flg and (i+1 > prev_day):
            continue
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        try:
            dates_objects[i].click()
        except:
            dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
            element = dates_objects[i]
            driver.execute_script("arguments[0].click();", element)
        # BC switcher
        for bc in bcs:
            sleep(1)     
            try:
                driver.find_elements(By.ID, bc)[0].click()
            except ElementClickInterceptedException:
                element = driver.find_elements(By.ID, bc)[0]
                driver.execute_script("arguments[0].click();", element)
            sleep(1)
            # buttons = WebDriverWait(driver, 10).until(
            #     EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".pages_pag .pages"))
            # )
            # filtered_ids = [x.get_attribute('id') for x in buttons if bc.split('_')[1] in x.get_attribute('id')]
            # # pages iterating
            # for id_ in filtered_ids:
            #     button = driver.find_elements(By.ID, id_)[0]
            #     try:
            #         driver.execute_script("arguments[0].click();", button) 
            #     except (StaleElementReferenceException, WebDriverException):
            #         sleep(1)
            #         button = driver.find_elements(By.ID, id_)[0]
            #         driver.execute_script("arguments[0].click();", button) 
            html_source_code = driver.page_source
            html_soup = BeautifulSoup(html_source_code, 'html.parser')
            tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
            # events parsing
            events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
            events = tbody_data[events_indexes]
            data_events[bc].append([get_data_event(x) for x in events])
            # tables parsing
            tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
            data_tables[bc].append(get_data_table(tables))

            dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
            try:
                dates_objects[i].click()
            except:
                dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
                element = dates_objects[i]
                driver.execute_script("arguments[0].click();", element)    
            try:
                driver.find_elements(By.ID, bc)[0].click()
            except ElementClickInterceptedException:
                element = driver.find_elements(By.ID, bc)[0]
                driver.execute_script("arguments[0].click();", element)
        save_data(i, current_page_month, current_page_year, data_tables, data_events)
    first_month_flg = False
    # month switcher
    left_triangle = driver.find_elements(By.LINK_TEXT, 'Назад')
    left_triangle[0].click()
    current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
    current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text

In [37]:
list(map(len, data_tables[bcs[0]])), list(map(len, data_events[bcs[0]]))

([50, 212, 205], [50, 212, 205])

In [63]:
# путь к спасению
element = driver.find_element(By.ID, f"response_{bc.split('_')[1]}")
temp_html = element.get_attribute("outerHTML")

In [65]:
temp_html

'<div id="response_1x" style="opacity: 1;"><header class="entry-header"><h3 class="entry_title">Движение линий в 1xstavka за 22-06-2023. Теннис</h3></header><meta http-equiv="Content-Type" content="text/html; charset=cp1251"><div id="#tennis"> </div><h3 style="border:1px solid black; background:orange;">Теннис\t\t   (212 матчей на Дату: 22.06.2023 г.)</h3><table class="liga"><tbody><tr><td>ATP. Галле</td></tr></tbody></table><table class="event"><tbody><tr><td>22.06.2023 13:05   Лоренцо Сонего - Янник Синнер;      (ID=25)</td> <td> Теннис :  ATP. Галле </td> </tr> <tr><td colspan="2">Итог матча:\t   \t Счет: 1:2 (7:6,4:6,4:6),  Победа: Янник Синнер,\tТотал Матча: 33,\t (ИТ1=15,  ИТ2=18,\tD=3) </td></tr> </tbody></table><table>   <tbody><tr> <th> Дата-время скана линии </th><th> П1 </th><th> П2 </th><th> 1Х </th><th> 2Х </th><th> Х </th><th> 12 </th><th> ТМ </th><th> ТМ бол </th><th> ТМ мен </th><th> ФОРА 1 </th><th> Коэф Ф1 </th><th> ФОРА 2 </th><th> Коэф Ф2 </th><th> Инд ТОТ 1 </th><t